# ASTR 19 Final Project

Import packages needed for the project

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.visualization import make_lupton_rgb
import sep
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Ellipse

Set `rcParams` for the format of the plots when using `matplotlib`

In [ ]:
rcParams["figure.figsize"] = [10., 8.]

Open the file as an HDU List object. Use `info()` to check the contents of the HDU List.

In [ ]:
hdul = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")
hdul.info()

Extract the data we are using from the HDU List.

In [ ]:
data = hdul[0].data

Plot the data using the `matplotlib` library.

In [ ]:
mean, std = np.mean(data), np.std(data)
plt.imshow(data, interpolation="nearest", cmap="gray", vmin=(mean - std), vmax=(mean + std), origin="lower")
plt.colorbar()

# save the figure as a png
plt.savefig("hubble_01.png", bbox_inches='tight', dpi=400)

Byte-swap operation in-place in order to use `sep` when analyzing the `.fits` file with `astropy.io.fits`. This is necessary because `astropy.io.fits` returns big-endian byte order arrays even on little-endian machines.

In [ ]:
data = data.byteswap(inplace=True).newbyteorder()

Store the `Background` object in the `bg` variable.

In [ ]:
bg = sep.Background(data)

Get the 2-d `numpy` array representation of the `Background` object.

In [ ]:
bg_img = bg.back()

Plot the background with `matplotlib`.

In [ ]:
plt.imshow(bg_img, interpolation="nearest", cmap="gray", origin="lower")
plt.colorbar()

# save the figure as a png
plt.savefig("hubble_02.png", bbox_inches='tight', dpi=400)

Create an array fo the background rms and store it in the `bg_rms` variable.

In [ ]:
bg_rms = bg.rms()

Plot the background noise with `matplotlib`.

In [ ]:
plt.imshow(bg_rms, interpolation="nearest", cmap="gray", origin="lower")
plt.colorbar()

# save the figure as a png
plt.savefig("hubble_03.png", bbox_inches='tight', dpi=400)

Subtract the background from the data.

In [ ]:
data_sub = data - bg

I can change the 2nd argument to adjust the threshhold for the objects that are found. Keeping it at 1.5 like in the tutorial resulted in too many objects being found because of the background noise.

In [ ]:
objects = sep.extract(data_sub, 15, err=bg.globalrms)

The number of objects found.

In [ ]:
print(f"Number of Sources: {len(objects)}")

Plot the graph with ellipses around the objects found using `matplotlib`.

In [ ]:
# background-subtracted image
fig, ax = plt.subplots()
mean, std = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation="nearest", cmap="gray", vmin=(mean - std), vmax=(mean + std), origin="lower")

# plot an ellipse for each object found
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), width=(6 * objects['a'][i]), height=(6 * objects['b'][i]), angle=(objects['theta'][i] * 180. / np.pi))
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

# save the figure as a png
plt.savefig("hubble_04.png", bbox_inches='tight', dpi=400)

Get the fluxes and the error values for each of those fluxes

In [ ]:
flux, flux_err, _ = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bg.globalrms, gain=1.0)

Show the first 10 objects' fluxes

In [ ]:
for i in range(10):
    print(f"object {i}: flux = {flux[i]} +/- {flux_err[i]}")

Plot the histogram of the fluxes using `matplotlib`.

In [ ]:
# define figure and axis
f, ax = plt.subplots(1, 1, figsize=(5, 5))

# create histogram
w = 25
ax.hist(
        flux,
        bins=np.arange((int)(min(flux)), (int)(max(flux) + w) + 1, w),
        edgecolor="black",
    )

# label axes
ax.set_xlabel("flux", fontsize=12)
ax.set_ylabel("count", fontsize=12)

# save figure as PDF
plt.savefig("hubble_05.png", bbox_inches='tight', dpi=400)

Mean, median, and standard deviation of the distribution of the fluxes

In [ ]:
mean = np.mean(flux)
median = np.median(flux)
std = np.std(flux)

print(f"Mean: {mean}\nMedian: {median}\nStandard Deviation: {std}")

Finding the largest outlier in the distribution

In [ ]:
outliers = [val for val in flux if np.abs(val - mean) > std * 3]
largest_outlier = max(outliers)

print(f"Largest outlier: {largest_outlier}")

Circle the largest outlier on the image

In [ ]:
# get the index of the outlier object
outlier_object_index = np.where(flux == largest_outlier)

# background-subtracted image
fig, ax = plt.subplots()
mean, std = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation="nearest", cmap="gray", vmin=(mean - std), vmax=(mean + std), origin="lower")

# draw a circle around the object on the image
e = Ellipse(xy=(objects['x'][outlier_object_index], objects['y'][outlier_object_index]), width=(6 * objects['a'][outlier_object_index]), height=(6 * objects['b'][outlier_object_index]), angle=(objects['theta'][outlier_object_index] * 180. / np.pi))
e.set_facecolor('none')
e.set_edgecolor('red')
ax.add_artist(e)

# save the figure as a png
plt.savefig("hubble_06.png")

The next code block calculates how many standard deviations the outlier is away from the mean.

In [ ]:
stds_from_mean = np.abs(largest_outlier - mean) / std

print(f"Largest outier is {stds_from_mean} stds away from the mean")

#### 3-color false image

Resources used:

- https://docs.astropy.org/en/stable/visualization/rgb.html

Open the f125w image as an HDU List and get the data

In [ ]:
hdul2 = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits")
# hdul2.info()
data2 = hdul2[0].data

Open the f160w image as an HDU List and get the data

In [ ]:
hdul3 = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits")
# hdul3.info()
data3 = hdul3[0].data

Create the 3-color false image of the UDFs

In [ ]:
# make the r-color false image
image = make_lupton_rgb(data3, data2, data, Q=0, stretch=0.01)

# plot the image
plt.imshow(image, origin="lower")

# save the figure as a png
plt.savefig("hubble_07.png", bbox_inches='tight', dpi=400)

Close the HDU List object when done using it

In [ ]:
hdul.close()
hdul2.close()
hdul3.close()